# Improved UNet pipepline with larger dataset - Copied Notebook with Modifications

#### [<u>Initial version</u>](https://www.kaggle.com/code/egortrushin/gwi-improved-unet-pipepline-with-larger-dataset)

- We used UNet model as introduced in [<u>5 depth U net with residual</u>](https://www.kaggle.com/code/adhok93/5-depth-u-net-with-residual) Notebook.
- We used part of Full OpenWFI dataset, which was introduced in [<u>OpenFWI InversionNet Train with 670G Datasets</u>](https://www.kaggle.com/code/seshurajup/openfwi-inversionnet-train-with-670g-datasets) Notebook.

#### <u>Copied version</u>

- We will use openFWI dataset converted from float32 to float16 ([<u>openfwi_float16_1</u>](https://www.kaggle.com/datasets/egortrushin/open-wfi-1), [<u>openfwi_float16_2</u>](https://www.kaggle.com/datasets/egortrushin/open-wfi-2), and [<u>openfwi_float16_test</u>](https://www.kaggle.com/datasets/egortrushin/open-wfi-test)). Since reading of data from disk is a bottleneck here, the conversion allows us to read data from disk twice faster and to use twice more data in training for the same runtime.

#### <u>Added Processing version</u>

- I have added denoising and normalization to see if 

### Config

In [ ]:
%%writefile config.yaml

data_path: /input
model: 
    name: UNet
    unet_params:
        init_features: 32
        depth: 5
read_weights: null
batch_size: 32
print_freq: 1000
max_epochs: 30
es_epochs: 4
seed: 42
valid_frac: 16
train_frac: 2
optimizer:
    lr: 0.0005
    weight_decay: 0.001
scheduler:
    params:
        factor: 0.316227766
        patience: 1

### Libraries

In [ ]:
# Standard library imports
import csv
import datetime
import os
import random
import sys
import time
import yaml
import gc
import shutil

# Scientific computing
import numpy as np
from scipy import ndimage
import scipy.signal as signal
from scipy.signal import butter, filtfilt, sosfilt
import scipy.stats as stats

# Deep learning
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.multiprocessing as mp
mp.set_sharing_strategy('file_system')

# Utilities
from pathlib import Path
from tqdm.auto import tqdm

In [ ]:
# Data
def inputs_files_to_output_files(input_files):
    return [
        Path(str(f).replace('seis', 'vel').replace('data', 'model'))
        for f in input_files
    ]

def get_train_files(data_path):

    all_inputs = [
        f
        for f in
        Path(data_path).rglob('*.npy')
        if ('seis' in f.stem) or ('data' in f.stem)
    ]

    all_outputs = inputs_files_to_output_files(all_inputs)

    assert all(f.exists() for f in all_outputs)

    return all_inputs, all_outputs

class SeismicDataset(Dataset):
    def __init__(self, inputs_files, output_files, n_examples_per_file=500):
        assert len(inputs_files) == len(output_files)
        self.inputs_files = inputs_files
        self.output_files = output_files
        self.n_examples_per_file = n_examples_per_file

    def __len__(self):
        return len(self.inputs_files) * self.n_examples_per_file

    def __getitem__(self, idx):
        # Calculate file offset and sample offset within file
        file_idx = idx // self.n_examples_per_file
        sample_idx = idx % self.n_examples_per_file

        X = np.load(self.inputs_files[file_idx], mmap_mode='r')
        y = np.load(self.output_files[file_idx], mmap_mode='r')

        try:
            return X[sample_idx].copy(), y[sample_idx].copy()
        finally:
            del X, y


class TestDataset(Dataset):
    def __init__(self, test_files):
        self.test_files = test_files


    def __len__(self):
        return len(self.test_files)


    def __getitem__(self, i):
        test_file = self.test_files[i]

        return np.load(test_file), test_file.stem

In [ ]:
# Model
class ResidualDoubleConv(nn.Module):
    """(Convolution => [BN] => ReLU) * 2 + Residual Connection"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels

        # First convolution layer
        self.conv1 = nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(mid_channels)
        self.relu = nn.ReLU(inplace=True)

        # Second convolution layer
        self.conv2 = nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # Shortcut connection to handle potential channel mismatch
        if in_channels == out_channels:
            self.shortcut = nn.Identity()
        else:
            # Projection shortcut: 1x1 conv + BN to match output channels
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = x  # Store the input for the residual connection

        # First conv block
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        # Second conv block (without final ReLU yet)
        out = self.conv2(out)
        out = self.bn2(out)

        # Apply shortcut to the identity path
        identity_mapped = self.shortcut(identity)

        # Add the residual connection
        out += identity_mapped

        # Apply final ReLU
        out = self.relu(out)
        return out


class Up(nn.Module):
    """Upscaling then ResidualDoubleConv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()
        self.bilinear = bilinear

        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode="bilinear", align_corners=False)
            # Input to ResidualDoubleConv = channels from upsampled layer below + channels from skip connection
            # Output of ResidualDoubleConv = desired output channels for this decoder stage
            self.conv = ResidualDoubleConv(in_channels + out_channels, out_channels) # Use ResidualDoubleConv

        else: # Using ConvTranspose2d
            # ConvTranspose halves the channels: in_channels -> in_channels // 2
            self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
            # Input channels to ResidualDoubleConv
            conv_in_channels = in_channels // 2 # Channels after ConvTranspose
            skip_channels = out_channels       # Channels from skip connection
            total_in_channels = conv_in_channels + skip_channels
            self.conv = ResidualDoubleConv(total_in_channels, out_channels) # Use ResidualDoubleConv

    def forward(self, x1, x2):
        # x1 is the feature map from the layer below (needs upsampling)
        # x2 is the skip connection from the corresponding encoder layer
        x1 = self.up(x1)

        # Pad x1 if its dimensions don't match x2 after upsampling
        # Input is CHW
        diffY = x2.size(2) - x1.size(2)
        diffX = x2.size(3) - x1.size(3)

        # Pad format: (padding_left, padding_right, padding_top, padding_bottom)
        x1 = F.pad(
            x1, [diffX // 2, diffX - diffX // 2, diffY // 2, diffY - diffY // 2]
        )

        # Concatenate along the channel dimension
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)

class OutConv(nn.Module):
    """1x1 Convolution for the output layer"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

class UNet(nn.Module):
    """U-Net architecture implementation with Residual Blocks"""

    def __init__(
        self,
        n_channels=5,
        n_classes=1,
        init_features=32,
        depth=5, # number of pooling layers
        bilinear=True,
    ):
        super().__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear
        self.depth = depth

        self.initial_pool = nn.AvgPool2d(kernel_size=(14, 1), stride=(14, 1))

        # --- Encoder ---
        self.encoder_convs = nn.ModuleList() # Store conv blocks
        self.encoder_pools = nn.ModuleList() # Store pool layers

        # Initial conv block (no pooling before it)
        # Use ResidualDoubleConv for the initial convolution block
        self.inc = ResidualDoubleConv(n_channels, init_features)
        self.encoder_convs.append(self.inc)

        current_features = init_features
        for _ in range(depth):
            # Define convolution block for this stage
            conv = ResidualDoubleConv(current_features, current_features * 2)
            # Define pooling layer for this stage
            pool = nn.MaxPool2d(2)
            self.encoder_convs.append(conv)
            self.encoder_pools.append(pool)
            current_features *= 2

        # --- Bottleneck ---
        # Use ResidualDoubleConv for the bottleneck
        self.bottleneck = ResidualDoubleConv(current_features, current_features)

        # --- Decoder ---
        self.decoder_blocks = nn.ModuleList()
        # Input features start from bottleneck output features
        # Output features at each stage are halved
        for _ in range(depth):
            # Up block uses ResidualDoubleConv internally and handles channels
            up_block = Up(current_features, current_features // 2, bilinear)
            self.decoder_blocks.append(up_block)
            current_features //= 2 # Halve features for next Up block input

        # --- Output Layer ---
        # Input features are the output features of the last Up block
        self.outc = OutConv(current_features, n_classes)

    def _pad_or_crop(self, x, target_h=70, target_w=70):
        """Pads or crops input tensor x to target height and width."""
        _, _, h, w = x.shape
        # Pad Height if needed
        if h < target_h:
            pad_top = (target_h - h) // 2
            pad_bottom = target_h - h - pad_top
            x = F.pad(x, (0, 0, pad_top, pad_bottom))  # Pad height only
            h = target_h
        # Pad Width if needed
        if w < target_w:
            pad_left = (target_w - w) // 2
            pad_right = target_w - w - pad_left
            x = F.pad(x, (pad_left, pad_right, 0, 0))  # Pad width only
            w = target_w
        # Crop Height if needed
        if h > target_h:
            crop_top = (h - target_h) // 2
            # Use slicing to crop
            x = x[:, :, crop_top : crop_top + target_h, :]
            h = target_h
        # Crop Width if needed
        if w > target_w:
            crop_left = (w - target_w) // 2
            x = x[:, :, :, crop_left : crop_left + target_w]
            w = target_w
        return x

    def forward(self, x):
        # Initial pooling and resizing
        x_pooled = self.initial_pool(x)
        x_resized = self._pad_or_crop(x_pooled, target_h=70, target_w=70)

        # --- Encoder Path ---
        skip_connections = []
        xi = x_resized

        # Apply initial conv (inc)
        xi = self.encoder_convs[0](xi)
        skip_connections.append(xi) # Store output of inc

        # Apply subsequent encoder convs and pools
        # self.depth is the number of pooling layers
        for i in range(self.depth):
            # Apply conv block for this stage
            xi = self.encoder_convs[i+1](xi)
            # Store skip connection *before* pooling
            skip_connections.append(xi)
            # Apply pooling layer for this stage
            xi = self.encoder_pools[i](xi)

        # Apply bottleneck conv
        xi = self.bottleneck(xi)

        # --- Decoder Path ---
        xu = xi # Start with bottleneck output
        # Iterate through decoder blocks and corresponding skip connections in reverse
        for i, block in enumerate(self.decoder_blocks):
            # Determine the correct skip connection index from the end
            # Example: depth=5. Skips stored: [inc, enc1, enc2, enc3, enc4] (indices 0-4)
            # Decoder 0 (Up(1024, 512)) needs skip 4 (enc4)
            # Decoder 1 (Up(512, 256)) needs skip 3 (enc3) ...
            # Decoder 4 (Up(64, 32)) needs skip 0 (inc)
            skip_index = self.depth - 1 - i
            skip = skip_connections[skip_index]
            xu = block(xu, skip) # Up block combines xu (from below) and skip

        # --- Final Output ---
        logits = self.outc(xu)
        # Apply scaling and offset specific to the problem's target range
        output = logits * 1000.0 + 1500.0
        return output

### Denoising

In [ ]:
# Helper functions 
def butter_bandpass(lowcut, highcut, fs, order=4):
    """Design a Butterworth bandpass filter."""
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band', output='ba')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=4):
    """Apply a Butterworth bandpass filter to a 1D signal."""
    b, a = butter_bandpass(lowcut, highcut, fs, order)
    y = filtfilt(b, a, data)
    return y

def denoise_seismic_with_bandpass(data, lowcut=0.004, highcut=0.3, fs=1.2, order=4):
    """
    Apply Butterworth bandpass filter to seismic data.
    Handles 2D, 3D, or 4D data.
    Inner tqdm progress bars are disabled to reduce verbosity.
    """
    original_shape = data.shape
    
    if len(original_shape) == 4: # 4D data (e.g., batch, channels, height, width)
        denoised_data = np.zeros_like(data)
        # Iterate through the first dimension (samples in batch)
        for s_idx in tqdm(range(original_shape[0]), desc="Processing samples in batch", leave=False, disable=True):
            for c_idx in range(original_shape[1]): # Channels
                denoised_data[s_idx, c_idx] = denoise_seismic_with_bandpass(
                    data[s_idx, c_idx], lowcut, highcut, fs, order
                )
        return denoised_data
    
    elif len(original_shape) == 3: # 3D data (e.g., batch/channels, height, width)
        denoised_data = np.zeros_like(data)
        # Iterate through the first dimension (channels or slices)
        for c_idx in tqdm(range(original_shape[0]), desc="Processing channels/slices", leave=False, disable=True):
            denoised_data[c_idx] = denoise_seismic_with_bandpass(
                data[c_idx], lowcut, highcut, fs, order
            )
        return denoised_data
    
    elif len(original_shape) == 2: # 2D data (height, width) - filter per trace
        denoised_data = np.zeros_like(data)
        for i in range(data.shape[1]): # Iterate over traces (width)
            trace = data[:, i]
            # filtfilt requirement: signal length > 3 * filter order
            if len(trace) > order * 3:
                 denoised_data[:, i] = butter_bandpass_filter(trace, lowcut, highcut, fs, order)
            else:
                # To avoid verbose output, this warning is kept as a comment.
                # print(f"Warning: Trace length ({len(trace)}) for trace {i} too short for filter order ({order}). Copying original trace.")
                denoised_data[:, i] = trace
        return denoised_data
    
    else:
        raise ValueError(f"Unsupported data shape: {original_shape}. Expected 2D, 3D, or 4D.")

# Revised memory-efficient function with reduced print output
def denoise_training_dataset_memory_efficient(train_loader, lowcut=0.004, highcut=0.3, fs=1.2, order=4):
    denoised_inputs_list = []
    targets_list = []

    print("Starting memory-efficient denoising process...")
    
    try:
        total_batches = len(train_loader)
    except TypeError:
        total_batches = None 

    overall_start_time = time.time()

    # This is the main progress bar the user will see, updating per batch
    for inputs_batch, targets_batch in tqdm(train_loader, desc="Denoising batches", total=total_batches, unit="batch"):
        inputs_batch_np = inputs_batch.numpy() 
        
        denoised_batch = denoise_seismic_with_bandpass(
            inputs_batch_np, lowcut=lowcut, highcut=highcut, fs=fs, order=order
        )

        denoised_inputs_list.append(denoised_batch)
        targets_list.append(targets_batch.numpy()) 
    
    overall_elapsed_time = time.time() - overall_start_time
    # Adding a newline to ensure this print is on a new line after tqdm finishes
    print(f"\nAll batches processed in {overall_elapsed_time:.2f} seconds.")

    print("Concatenating denoised batches and targets...")
    concatenation_start_time = time.time()
    
    final_denoised_inputs = np.concatenate(denoised_inputs_list, axis=0)
    final_targets = np.concatenate(targets_list, axis=0)
    
    concatenation_elapsed_time = time.time() - concatenation_start_time
    print(f"Concatenation completed in {concatenation_elapsed_time:.2f} seconds.")
    
    return final_denoised_inputs, final_targets

In [ ]:
# Create a custom dataset that applies denoising on-the-fly
class DenoisedSeismicDataset(Dataset):
    def __init__(self, inputs_files, output_files, n_examples_per_file=500, 
                 apply_denoising=True, lowcut=0.004, highcut=0.3, fs=1.2, order=4):
        assert len(inputs_files) == len(output_files)
        self.inputs_files = inputs_files
        self.output_files = output_files
        self.n_examples_per_file = n_examples_per_file
        self.apply_denoising = apply_denoising
        self.lowcut = lowcut
        self.highcut = highcut
        self.fs = fs
        self.order = order

    def __len__(self):
        return len(self.inputs_files) * self.n_examples_per_file

    def __getitem__(self, idx):
        # Calculate file offset and sample offset within file
        file_idx = idx // self.n_examples_per_file
        sample_idx = idx % self.n_examples_per_file

        X = np.load(self.inputs_files[file_idx], mmap_mode='r')
        y = np.load(self.output_files[file_idx], mmap_mode='r')

        try:
            input_data = X[sample_idx].copy()
            target_data = y[sample_idx].copy()
            
            # Apply denoising if enabled
            if self.apply_denoising:
                input_data = denoise_seismic_with_bandpass(
                    input_data, 
                    lowcut=self.lowcut, 
                    highcut=self.highcut, 
                    fs=self.fs, 
                    order=self.order
                )
                
            return input_data, target_data
        finally:
            del X, y

In [ ]:
# Function to check available disk space
def check_available_disk_space(required_gb, save_dir):
    """Check if there's enough disk space available."""
    os.makedirs(save_dir, exist_ok=True)
    
    disk_usage = shutil.disk_usage(save_dir)
    available_gb = disk_usage.free / (1024**3)
    
    print(f"Available disk space: {available_gb:.2f} GB")
    print(f"Required disk space: {required_gb:.2f} GB")
    
    return available_gb >= required_gb

# Function to denoise and save the dataset
def denoise_and_save_dataset(input_files, output_files, save_dir, 
                            lowcut=0.004, highcut=0.3, fs=1.2, order=4,
                            prefix="train"):
    """Process all files, apply denoising, and save to disk."""
    # Create directory for denoised data
    denoised_dir = os.path.join(save_dir, "denoised_data")
    os.makedirs(denoised_dir, exist_ok=True)
    
    # Estimate required disk space
    sample_input = np.load(input_files[0], mmap_mode='r')
    sample_size_bytes = sample_input.nbytes
    total_files = len(input_files)
    estimated_gb = (sample_size_bytes * total_files * 2) / (1024**3)  # x2 for inputs and targets
    
    # Check for sufficient disk space
    if not check_available_disk_space(estimated_gb, save_dir):
        raise RuntimeError(f"Not enough disk space. Need approximately {estimated_gb:.2f} GB")
    
    denoised_input_paths = []
    target_output_paths = []
    
    print(f"Processing {len(input_files)} files with prefix '{prefix}'...")
    
    # Process each file
    for i, (input_file, output_file) in enumerate(tqdm(zip(input_files, output_files), 
                                                      total=len(input_files),
                                                      desc=f"Denoising {prefix} files")):
        # Generate filenames
        base_name = os.path.basename(input_file)
        denoised_filename = f"{prefix}_denoised_{i}_{base_name}"
        target_filename = f"{prefix}_target_{i}_{os.path.basename(output_file)}"
        
        denoised_path = os.path.join(denoised_dir, denoised_filename)
        target_path = os.path.join(denoised_dir, target_filename)
        
        # Add to path lists
        denoised_input_paths.append(denoised_path)
        target_output_paths.append(target_path)
        
        # Skip if files already exist
        if os.path.exists(denoised_path) and os.path.exists(target_path):
            print(f"Files already exist for {base_name}, skipping")
            continue
        
        try:
            # Load data
            input_data = np.load(input_file)
            target_data = np.load(output_file)
            
            # Apply denoising
            denoised_input = denoise_seismic_with_bandpass(
                input_data, 
                lowcut=lowcut, 
                highcut=highcut, 
                fs=fs, 
                order=order
            )
            
            # Save processed data
            np.save(denoised_path, denoised_input)
            np.save(target_path, target_data)
            
            # Free memory
            del input_data, denoised_input, target_data
            gc.collect()
            
        except Exception as e:
            print(f"Error processing file {input_file}: {str(e)}")
            continue
    
    return denoised_input_paths, target_output_paths

# Dataset class for pre-denoised data
class PreDenoisedSeismicDataset(Dataset):
    def __init__(self, inputs_files, output_files, n_examples_per_file=250):
        assert len(inputs_files) == len(output_files)
        self.inputs_files = inputs_files
        self.output_files = output_files
        self.n_examples_per_file = n_examples_per_file

    def __len__(self):
        return len(self.inputs_files) * self.n_examples_per_file

    def __getitem__(self, idx):
        # Calculate which file and sample within file
        file_idx = idx // self.n_examples_per_file
        sample_idx = idx % self.n_examples_per_file

        try:
            # Use memory-mapped loading for efficiency
            X = np.load(self.inputs_files[file_idx], mmap_mode='r')
            y = np.load(self.output_files[file_idx], mmap_mode='r')
            
            # Copy to avoid memory issues with mmap
            input_data = X[sample_idx].copy()
            target_data = y[sample_idx].copy()
                
            return torch.from_numpy(input_data).float(), torch.from_numpy(target_data).float()
        except Exception as e:
            print(f"Error loading sample {idx} from file {self.inputs_files[file_idx]}: {str(e)}")
            # Return zeros with appropriate shape (adjust based on your data dimensions)
            return torch.zeros((1, 224, 224), dtype=torch.float32), torch.zeros((1, 224, 224), dtype=torch.float32)
        finally:
            # Clean up memory-mapped files
            if 'X' in locals():
                del X
            if 'y' in locals():
                del y

### Utilities

In [ ]:
# Utils
def format_time(elapsed):
    """Take a time in seconds and return a string hh:mm:ss."""
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

def seed_everything(
    seed_value: int
) -> None:

    random.seed(seed_value) # Python
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
    if torch.backends.cudnn.is_available:
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

### Preparation and Training

In [ ]:
# Load configuration
print(f"GPU: {torch.cuda.get_device_name(0)}")
_, total = torch.cuda.mem_get_info(device=0)
print(f"GPU memory: {total / 1024**3:.2f}GB")

with open("config.yaml", "r") as file_obj:
    config = yaml.safe_load(file_obj)
print()
print(config)
if config["data_path"] is None:
    config["data_path"] = os.environ["TMPDIR"]
    print("data_path:", config["data_path"])
print()

In [ ]:
# Preparation and training
# Refactored training pipeline with pre-denoising
# Set random seed
seed_everything(config["seed"])

# Get data files
all_inputs, all_outputs = [], []
for x in ["input/train_samples", "input/openfwi_float16_1", "input/openfwi_float16_2"]:
    all_inputs1, all_outputs1 = get_train_files(x)
    all_inputs.extend(all_inputs1)
    all_outputs.extend(all_outputs1)
print("Total number of input/output files:", len(all_inputs))

# Split into train and validation sets
valid_indices = list(range(0, len(all_inputs), config["valid_frac"]))
valid_inputs = [all_inputs[i] for i in valid_indices]
train_inputs = [f for f in all_inputs if f not in valid_inputs]

if config["train_frac"] > 1:
    train_indices = list(range(0, len(train_inputs), config["train_frac"]))
    train_inputs = [train_inputs[i] for i in train_indices]

print("Number of train files:", len(train_inputs))
print("Number of valid files:", len(valid_inputs))

train_outputs = inputs_files_to_output_files(train_inputs)
valid_outputs = inputs_files_to_output_files(valid_inputs)

## Channel Adaptive Processing

In [ ]:
# Refactored processing functions with improved efficiency and error handling
def histogram_equalization_seismic(data, n_bins=1000, clip_percentile=99.5):
    """
    Apply histogram equalization to seismic data to improve amplitude distribution.
    Simplified to work on single samples rather than batches.
    """
    # Get shape information
    if data.ndim == 3:  # Single sample: (channels, time, traces)
        n_channels, time_steps, n_traces = data.shape
        equalized_data = np.zeros_like(data)
        
        for c in range(n_channels):
            channel_data = data[c]
            
            # Find non-zero values (signal)
            non_zero_mask = np.abs(channel_data) > 1e-6
            if np.sum(non_zero_mask) > 100:
                # Get data values for histogram computation
                data_values = channel_data[non_zero_mask]
                
                # Clip extreme outliers
                clip_value = np.percentile(np.abs(data_values), clip_percentile)
                clipped_data = np.clip(channel_data, -clip_value, clip_value)
                
                # Create histogram and cumulative distribution
                hist, bin_edges = np.histogram(clipped_data, bins=n_bins, range=(-clip_value, clip_value))
                cdf = hist.cumsum() / (hist.sum() + 1e-10)
                
                # Create mapping function
                bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
                equalized = np.interp(clipped_data.flatten(), bin_centers, 2 * cdf - 1).reshape(clipped_data.shape)
                
                # Apply mild smoothing
                equalized = ndimage.gaussian_filter(equalized, sigma=0.5)
                equalized_data[c] = np.clip(equalized, -1, 1)
            else:
                equalized_data[c] = data[c]
                
        return equalized_data
    else:
        raise ValueError(f"Expected 3D array, got {data.ndim}D")

def adaptive_agc_single(data, window_sizes=[50, 100, 200], blend_weights=[0.2, 0.3, 0.5]):
    """
    Apply multi-scale AGC to a single sample.
    """
    if data.ndim == 3:  # (channels, time, traces)
        n_channels, time_steps, n_traces = data.shape
        balanced_data = np.zeros_like(data)
        
        # Normalize weights
        blend_weights = np.array(blend_weights) / np.sum(blend_weights)
        
        for c in range(n_channels):
            for t in range(n_traces):
                trace = data[c, :, t]
                
                # Skip if trace is mostly zeros
                if np.sum(np.abs(trace)) < 1e-6:
                    balanced_data[c, :, t] = trace
                    continue
                
                blended_trace = np.zeros_like(trace)
                
                # Apply AGC at multiple scales
                for i, window_size in enumerate(window_sizes):
                    half_win = window_size // 2
                    agc_trace = np.zeros_like(trace)
                    
                    for i_sample in range(time_steps):
                        win_start = max(0, i_sample - half_win)
                        win_end = min(time_steps, i_sample + half_win + 1)
                        
                        # Calculate RMS in window
                        window_rms = np.sqrt(np.mean(trace[win_start:win_end]**2) + 1e-10)
                        agc_trace[i_sample] = trace[i_sample] / window_rms
                    
                    blended_trace += blend_weights[i] * agc_trace
                
                # Normalize
                max_abs = np.max(np.abs(blended_trace))
                if max_abs > 1.0:
                    blended_trace /= max_abs
                    
                balanced_data[c, :, t] = blended_trace
        
        # Apply final smoothing
        for c in range(n_channels):
            balanced_data[c] = ndimage.gaussian_filter(balanced_data[c], sigma=0.5)
            
        return np.clip(balanced_data, -1.0, 1.0)
    else:
        raise ValueError(f"Expected 3D array, got {data.ndim}D")

def amplitude_distribution_matching_single(data, alpha=0.4, smoothing_sigma=0.5):
    """
    Apply amplitude distribution matching to a single sample.
    """
    if data.ndim == 3:  # (channels, time, traces)
        n_channels, time_steps, n_traces = data.shape
        matched_data = np.copy(data)
        
        for c in range(n_channels):
            channel_data = data[c].flatten()
            
            # Find non-zero values
            non_zero_mask = np.abs(channel_data) > 0.01
            if np.sum(non_zero_mask) > 100:
                channel_values = channel_data[non_zero_mask]
                
                # Calculate contrast enhancement
                enhancement = np.sign(channel_values) * (np.abs(channel_values) ** alpha)
                
                # Create mapping
                sorted_orig = np.sort(channel_values)
                sorted_enhanced = np.sort(enhancement)
                
                # Apply mapping to each trace
                for t in range(n_traces):
                    trace = matched_data[c, :, t]
                    mapped_trace = np.interp(trace, sorted_orig, sorted_enhanced)
                    
                    if smoothing_sigma > 0:
                        mapped_trace = ndimage.gaussian_filter1d(mapped_trace, sigma=smoothing_sigma)
                    
                    matched_data[c, :, t] = mapped_trace
        
        # Normalize
        for c in range(n_channels):
            max_abs = np.max(np.abs(matched_data[c]))
            if max_abs > 1.0:
                matched_data[c] /= max_abs
                
        return np.clip(matched_data, -1.0, 1.0)
    else:
        raise ValueError(f"Expected 3D array, got {data.ndim}D")

def apply_channel_adaptive_processing(data):
    """
    Apply different processing strategies for different channels.
    """
    if data.ndim != 3:
        raise ValueError(f"Expected 3D array (channels, time, traces), got {data.ndim}D")
        
    n_channels = data.shape[0]
    processed_data = np.copy(data)
    
    # Define channel groups
    linear_channels = [0, 4]
    reflection_channels = [1, 2, 3]
    
    # Process linear channels with AGC
    for c in linear_channels:
        if c < n_channels:
            channel_data = processed_data[c:c+1]
            processed_channel = adaptive_agc_single(
                channel_data,
                window_sizes=[30, 100, 200],
                blend_weights=[0.3, 0.4, 0.3]
            )
            processed_data[c] = processed_channel[0]
    
    # Process reflection channels with distribution matching
    for c in reflection_channels:
        if c < n_channels:
            channel_data = processed_data[c:c+1]
            processed_channel = amplitude_distribution_matching_single(
                channel_data,
                alpha=0.4,
                smoothing_sigma=0.5
            )
            processed_data[c] = processed_channel[0]
    
    return processed_data

In [ ]:
# Add this new function for batch processing with progress tracking
def preprocess_dataset_with_progress(inputs_files, output_files, processing_type='adaptive', 
                                    n_examples_per_file=500, save_preprocessed=False, save_dir=None):
    """
    Preprocess entire dataset with overall progress tracking.
    
    Parameters:
    -----------
    inputs_files : list
        List of input file paths
    output_files : list
        List of output file paths
    processing_type : str
        Type of processing: 'adaptive', 'histogram', 'agc', 'distribution'
    n_examples_per_file : int
        Number of examples per file
    save_preprocessed : bool
        Whether to save preprocessed data to disk
    save_dir : str
        Directory to save preprocessed data (if save_preprocessed=True)
        
    Returns:
    --------
    tuple
        (processed_inputs_list, targets_list) or (processed_input_paths, target_paths) if saved
    """
    total_samples = len(inputs_files) * n_examples_per_file
    processed_inputs_list = []
    targets_list = []
    processed_input_paths = []
    target_paths = []
    
    print(f"Starting {processing_type} processing for {total_samples} samples...")
    
    # Create overall progress bar
    with tqdm(total=total_samples, desc=f"Applying {processing_type} processing") as pbar:
        
        for file_idx, (input_file, output_file) in enumerate(zip(inputs_files, output_files)):
            # Load the entire file
            X = np.load(input_file)
            y = np.load(output_file)
            
            if save_preprocessed and save_dir:
                # Create arrays to hold processed data for this file
                processed_X = np.zeros_like(X)
                
            for sample_idx in range(min(n_examples_per_file, len(X))):
                input_data = X[sample_idx].copy()
                target_data = y[sample_idx].copy()
                
                # Normalize the input data
                input_mean = np.mean(input_data)
                input_std = np.std(input_data) + 1e-8
                normalized_input = (input_data - input_mean) / input_std
                
                # Apply selected processing
                if processing_type == 'adaptive':
                    processed_input = apply_channel_adaptive_processing(normalized_input)
                elif processing_type == 'histogram':
                    processed_input = histogram_equalization_seismic(normalized_input)
                elif processing_type == 'agc':
                    processed_input = adaptive_agc_single(normalized_input)
                elif processing_type == 'distribution':
                    processed_input = amplitude_distribution_matching_single(normalized_input)
                else:
                    processed_input = normalized_input
                
                if save_preprocessed and save_dir:
                    processed_X[sample_idx] = processed_input
                else:
                    processed_inputs_list.append(processed_input)
                    targets_list.append(target_data)
                
                # Update progress bar
                pbar.update(1)
            
            # Save processed file if requested
            if save_preprocessed and save_dir:
                os.makedirs(save_dir, exist_ok=True)
                base_name = os.path.basename(input_file)
                processed_filename = f"processed_{processing_type}_{base_name}"
                target_filename = f"target_{os.path.basename(output_file)}"
                
                processed_path = os.path.join(save_dir, processed_filename)
                target_path = os.path.join(save_dir, target_filename)
                
                np.save(processed_path, processed_X[:n_examples_per_file])
                np.save(target_path, y[:n_examples_per_file])
                
                processed_input_paths.append(processed_path)
                target_paths.append(target_path)
                
            # Free memory
            del X, y
            if save_preprocessed and save_dir:
                del processed_X
            gc.collect()
    
    print(f"Processing completed!")
    
    if save_preprocessed and save_dir:
        return processed_input_paths, target_paths
    else:
        return processed_inputs_list, targets_list

In [ ]:
# Create a custom dataset that applies channel adaptive processing on-the-fly
# Modified dataset class with optional preprocessing
class ChannelAdaptiveSeismicDataset(Dataset):
    def __init__(self, inputs_files, output_files, n_examples_per_file=500, 
                 apply_processing=True, processing_type='adaptive', 
                 preprocess_all=False, cache_processed=False):
        """
        Dataset with channel adaptive processing.
        
        Parameters:
        -----------
        processing_type : str
            Type of processing: 'adaptive', 'histogram', 'agc', 'distribution', or None
        preprocess_all : bool
            If True, preprocess all data at initialization with progress bar
        cache_processed : bool
            If True and preprocess_all=True, cache processed data in memory
        """
        assert len(inputs_files) == len(output_files)
        self.inputs_files = inputs_files
        self.output_files = output_files
        self.n_examples_per_file = n_examples_per_file
        self.apply_processing = apply_processing
        self.processing_type = processing_type
        self.cache_processed = cache_processed
        self.processed_cache = {}
        
        # Preprocess all data if requested
        if preprocess_all and apply_processing and processing_type:
            self._preprocess_all_data()

    def _preprocess_all_data(self):
        """Preprocess all data with progress tracking."""
        total_samples = len(self.inputs_files) * self.n_examples_per_file
        
        print(f"Preprocessing {total_samples} samples with {self.processing_type} processing...")
        
        with tqdm(total=total_samples, desc=f"Preprocessing dataset") as pbar:
            for file_idx, (input_file, output_file) in enumerate(zip(self.inputs_files, self.output_files)):
                X = np.load(input_file)
                y = np.load(output_file)
                
                for sample_idx in range(min(self.n_examples_per_file, len(X))):
                    global_idx = file_idx * self.n_examples_per_file + sample_idx
                    
                    input_data = X[sample_idx].copy()
                    target_data = y[sample_idx].copy()
                    
                    # Process the data
                    processed_input = self._process_sample(input_data)
                    
                    if self.cache_processed:
                        self.processed_cache[global_idx] = (processed_input, target_data)
                    
                    pbar.update(1)
                
                del X, y
                gc.collect()
        
        print("Preprocessing completed!")

    def _process_sample(self, input_data):
        """Process a single sample."""
        # Normalize the input data
        input_mean = np.mean(input_data)
        input_std = np.std(input_data) + 1e-8
        normalized_input = (input_data - input_mean) / input_std
        
        # Apply selected processing
        if self.processing_type == 'adaptive':
            processed_input = apply_channel_adaptive_processing(normalized_input)
        elif self.processing_type == 'histogram':
            processed_input = histogram_equalization_seismic(normalized_input)
        elif self.processing_type == 'agc':
            processed_input = adaptive_agc_single(normalized_input)
        elif self.processing_type == 'distribution':
            processed_input = amplitude_distribution_matching_single(normalized_input)
        else:
            processed_input = normalized_input
            
        return processed_input

    def __len__(self):
        return len(self.inputs_files) * self.n_examples_per_file

    def __getitem__(self, idx):
        # Check cache first
        if idx in self.processed_cache:
            return self.processed_cache[idx]
        
        # Calculate file offset and sample offset within file
        file_idx = idx // self.n_examples_per_file
        sample_idx = idx % self.n_examples_per_file

        X = np.load(self.inputs_files[file_idx], mmap_mode='r')
        y = np.load(self.output_files[file_idx], mmap_mode='r')

        try:
            input_data = X[sample_idx].copy()
            target_data = y[sample_idx].copy()
            
            # Apply processing if enabled and not already processed
            if self.apply_processing and self.processing_type and idx not in self.processed_cache:
                input_data = self._process_sample(input_data)
                
            return input_data, target_data
        finally:
            del X, y

### Denoise Params

In [ ]:
# Set parameters for denoising
denoise_params = {
    "lowcut": 0.002,
    "highcut": 0.3,
    "fs": 1.2,
    "order": 4
}

# Create directory for saving denoised data
save_dir = config.get("denoised_save_dir", "denoised_data")
os.makedirs(save_dir, exist_ok=True)

In [ ]:
# Process and save denoised training data
#print("\nPreprocessing and saving denoised training data...")
#denoised_train_inputs, denoised_train_outputs = denoise_and_save_dataset(
#    train_inputs, 
#    train_outputs, 
#    save_dir=save_dir,
#    prefix="train", 
#    **denoise_params
#)

# Process and save denoised validation data
#print("\nPreprocessing and saving denoised validation data...")
#denoised_valid_inputs, denoised_valid_outputs = denoise_and_save_dataset(
#    valid_inputs, 
#    valid_outputs, 
#    save_dir=save_dir,
#    prefix="valid", 
#    **denoise_params
#)


In [ ]:
# Preprocess and save to disk
processed_save_dir = "preprocessed_adaptive"

print("Preprocessing training data...")
processed_train_inputs, processed_train_outputs = preprocess_dataset_with_progress(
    train_inputs, 
    train_outputs,
    processing_type='adaptive',
    n_examples_per_file=500,
    save_preprocessed=True,
    save_dir=os.path.join(processed_save_dir, "train")
)

print("\nPreprocessing validation data...")
processed_valid_inputs, processed_valid_outputs = preprocess_dataset_with_progress(
    valid_inputs, 
    valid_outputs,
    processing_type='adaptive',
    n_examples_per_file=500,
    save_preprocessed=True,
    save_dir=os.path.join(processed_save_dir, "valid")
)

# Then use regular SeismicDataset with preprocessed files
dstrain = SeismicDataset(processed_train_inputs, processed_train_outputs)
dsvalid = SeismicDataset(processed_valid_inputs, processed_valid_outputs)

In [ ]:
# Create datasets using pre-denoised data
#dstrain = PreDenoisedSeismicDataset(
#    denoised_train_inputs, 
#    denoised_train_outputs,
#    n_examples_per_file=config.get("n_examples_per_file", 500)
#)

#dsvalid = PreDenoisedSeismicDataset(
#    denoised_valid_inputs, 
#    denoised_valid_outputs,
#    n_examples_per_file=config.get("n_examples_per_file", 500)
#)

# Load seismic
#dstrain = SeismicDataset(train_inputs, train_outputs)
#dsvalid = SeismicDataset(valid_inputs, valid_outputs)

# Create datasets with channel adaptive processing for training
dstrain = ChannelAdaptiveSeismicDataset(
    train_inputs, 
    train_outputs, 
    apply_processing=True,
    processing_type='adaptive'  # Options: 'adaptive', 'histogram', 'agc', 'distribution', None
)

# For validation, you can choose whether to apply processing
dsvalid = ChannelAdaptiveSeismicDataset(
    valid_inputs, 
    valid_outputs, 
    apply_processing=False  # Usually don't process validation data
)


# Create data loaders
dltrain = DataLoader(
    dstrain,
    batch_size=config["batch_size"],
    shuffle=True,  # Can shuffle now that we're not processing on-the-fly
    pin_memory=config.get("pin_memory", False),
    drop_last=True,
    num_workers=config.get("num_workers", 0),
    persistent_workers=config.get("persistent_workers", False),
)

dlvalid = DataLoader(
    dsvalid,
    batch_size=4*config["batch_size"],
    shuffle=False,
    pin_memory=config.get("pin_memory", False),
    drop_last=False,
    num_workers=config.get("num_workers", 0),
    persistent_workers=config.get("persistent_workers", False),
)

# Set up device, model, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet(**config["model"]["unet_params"]).to(device)

if config["read_weights"] is not None:
    print("Reading weights from:", config["read_weights"])
    model.load_state_dict(torch.load(config["read_weights"], weights_only=True))

criterion = nn.L1Loss()
optimizer = torch.optim.AdamW(model.parameters(), **config["optimizer"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', **config["scheduler"]["params"])

# Training loop
best_val_loss = 10000.0
epochs_wo_improvement = 0
t0 = time.time()

for epoch in range(1, config["max_epochs"] + 1):
    # Train
    model.train()
    train_losses = []
    
    for step, (inputs, targets) in enumerate(dltrain):
        inputs = inputs.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()

        with torch.autocast(device_type="cuda"):
            outputs = model(inputs)
            loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())

        if step % config["print_freq"] == config["print_freq"] - 1 or step == len(dltrain) - 1:
            trn_loss = np.mean(train_losses)
            t1 = format_time(time.time() - t0)
            free, total = torch.cuda.mem_get_info(device=0)
            mem_used = (total - free) / 1024**3
            lr = optimizer.param_groups[-1]['lr']
            print(
                f"Epoch: {epoch:02d}  Step {step+1}/{len(dltrain)}  Trn Loss: {trn_loss:.2f}  LR: {lr:.2e}  GPU Usage: {mem_used:.2f}GB  Elapsed Time: {t1}",
                flush=True,
            )

    # Validation
    model.eval()
    valid_losses = []
    
    for inputs, targets in dlvalid:
        inputs = inputs.to(device)
        targets = targets.to(device)

        with torch.inference_mode():
            with torch.autocast(device_type="cuda"):
                outputs = model(inputs)

        loss = criterion(outputs, targets)
        valid_losses.append(loss.item())

    t1 = format_time(time.time() - t0)
    trn_loss = np.mean(train_losses)
    val_loss = np.mean(valid_losses)

    free, total = torch.cuda.mem_get_info(device=0)
    mem_used = (total - free) / 1024**3

    print(
        f"\nEpoch: {epoch:02d}  Trn Loss: {trn_loss:.2f}  Val Loss: {val_loss:.2f}  GPU Usage: {mem_used:.2f}GB  Elapsed Time: {t1}",
        flush=True,
    )
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_wo_improvement = 0
        torch.save(model.state_dict(), "best_model.pth")
        print(f"\nNew best val_loss: {val_loss:.2f}\n", flush=True)
    else:
        epochs_wo_improvement += 1
        print(f"\nEpochs without improvement: {epochs_wo_improvement}\n", flush=True)

    if epochs_wo_improvement == config["es_epochs"]:
        break

    scheduler.step(val_loss)

print("\n" + "="*50)
print("MODEL TRAINING COMPLETE")
print("="*50)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

# Print model summary
print(f"\nModel Summary:")
print(f"Architecture: UNet")
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Best validation loss: {best_val_loss:.4f}")
print(f"Total training time: {format_time(time.time() - t0)}")

# Print model structure
print("\nModel Structure:")
print(model)
print("="*50)

torch.save(model.state_dict(), f"{best_val_loss:.4f}_model.pth")

# Create a summary text file
summary_filename = f"model_summary_{best_val_loss:.4f}.txt"

with open(summary_filename, 'w') as f:
    # Write header
    f.write("="*50 + "\n")
    f.write("MODEL TRAINING SUMMARY\n")
    f.write("="*50 + "\n\n")
    
    # Write model information
    f.write("MODEL INFORMATION\n")
    f.write(f"Architecture: UNet\n")
    f.write(f"Total parameters: {total_params:,}\n")
    f.write(f"Trainable parameters: {trainable_params:,}\n")
    f.write(f"Best validation loss: {best_val_loss:.4f}\n")
    f.write(f"Total training time: {format_time(time.time() - t0)}\n\n")
    
    # Write configuration information
    f.write("MODEL CONFIGURATION\n")
    for key, value in config.items():
        if isinstance(value, dict):
            f.write(f"{key}:\n")
            for sub_key, sub_value in value.items():
                f.write(f"  {sub_key}: {sub_value}\n")
        else:
            f.write(f"{key}: {value}\n")
    f.write("\n")
    
    # Write model structure
    f.write("MODEL STRUCTURE\n")
    f.write(str(model) + "\n")
    
    f.write("="*50 + "\n")

print(f"\nModel summary saved to {summary_filename}")

In [ ]:
# Load the best model
model.load_state_dict(torch.load("best_model.pth"))
model.eval()

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

# Print model summary
print(f"\nModel Summary:")
print(f"Architecture: UNet")
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Best validation loss: {best_val_loss:.4f}")
print(f"Total training time: {format_time(time.time() - t0)}")

# Print model structure
print("\nModel Structure:")
print(model)
print("="*50)

In [ ]:
print("==================== Inference ===============")

### Inference

In [ ]:
# Inference
t0 = time.time()

test_files = list(Path("input/openfwi_float16_test/test").glob("*.npy"))
x_cols = [f"x_{i}" for i in range(1, 70, 2)]
fieldnames = ["oid_ypos"] + x_cols
ds = TestDataset(test_files)
dl = DataLoader(ds, batch_size=4*config["batch_size"], num_workers=0, pin_memory=False)

model.load_state_dict(torch.load("best_model.pth", weights_only=True))

model.eval()
with open("submission.csv", "wt", newline="") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for inputs, oids_test in dl:
        inputs = inputs.to(device)
        with torch.inference_mode():
            with torch.autocast(device_type="cuda"):
                outputs = model(inputs)

        y_preds = outputs[:, 0].cpu().numpy()

        for y_pred, oid_test in zip(y_preds, oids_test):
            for y_pos in range(70):
                row = dict(zip(x_cols, [y_pred[y_pos, x_pos] for x_pos in range(1, 70, 2)]))
                row["oid_ypos"] = f"{oid_test}_y_{y_pos}"

                writer.writerow(row)

t1 = format_time(time.time() - t0)
print(f"Inference Time: {t1}")